I have placed some of the less interesting code in the `convert_book_to_text.py` file to try to keep the notebook flowing

In [2]:
##for the reload of all the functions in the file convert_pdf_to_text.py
from importlib import reload
import convert_book_to_text

reload(convert_book_to_text)
from convert_book_to_text import find_chapter_pages, \
                                 find_page_starting_with,\
                                 extract_text_from_pages_inclusive, \
                                 build_chapter_dictionary, \
                                 remove_chapter_header, \
                                 chunk_text, \
                                 find_bad_csv_files


For copyright reasons I have not included the ebook in the repo. You should buy your own copy if you want to run this code

In [3]:
from pathlib import Path
import PyPDF2

input_file = Path('./inputs/Truth to Power_ My Three Years Inside Eskom.pdf')
input_file = Path(input_file)
# Instantiate PdfReader object
pdf = PyPDF2.PdfReader(input_file)


In [4]:
# This is very specific to the format of the book I am precessing. Not making this general is a design decision. This repo needs back and forward with a human
# to work so spending too much time generalising functions seems like a waste of time.

# Note: This is quite slow. It takes about 30 seconds to run on my machine.
pages_with_numbers = find_chapter_pages(pdf)
pages_with_numbers.append(find_page_starting_with(pdf, "Index"))
# Create a dictionary where each chapter is saved with its start and end page
chapter_start_and_end = build_chapter_dictionary(pages_with_numbers)

The main loop. Chapters start at 1 and end at 31. chapter_range_end should be set to the number after the last chapter you want to process so, if you are processing the last chapter, chapter 31, only you would set `chapter_range_start = 31` and `chapter_range_end = 32`.

The output of this is a list of CSV files in the folder `./output/`. A separate file is created for each chunk. 

The function sets a chunk size of 800 words. There is no science in determining how long this should be but we need to ensure there are sufficient tokens for the input, the system command and the output. I started with a chunk size of 1000 words but I found there were times where GPT3.5 would truncate the output. 800 seemed reasonable. 

In [5]:
# from importlib import reload
# import summarise

# reload(summarise)
# from summarise import save_people_named_in_chapters, \
#                                combine_csvs, \
#                                aggregate_sentiment

ImportError: cannot import name 'save_people_named_in_chapters' from 'summarise' (e:\Code\pdf\ebook\summarise.py)

In [27]:
import openai

output_folder = "./summary_details"
# create the Path object and the folder if it doesn't exist
folder_path = Path(output_folder)
folder_path.mkdir(parents=True, exist_ok=True)
#folder_path = Path(output_folder)

chapter_range_start = 1
chapter_range_end = 2

system_prompt = "You will be presented with an extract from a book. You need to summarise this text into a short paragraph."

for i in range(chapter_range_start, chapter_range_end):
        chapter_number = i
        start = chapter_start_and_end[i]["start_page"]
        end = chapter_start_and_end[i]["end_page"]

        print("processing chapter: ", str(chapter_number) + " from page " + str(start) + " to page " + str(end))
        text = remove_chapter_header(extract_text_from_pages_inclusive(pdf, start, end))
        chunked_text = chunk_text(text, 1000)

        for chunk in chunked_text:
            #create a filename using folder_pah + "chapter" + chapter_number + "_" + index of chunk + ".csv"
            filename = folder_path / ("chapter_" + str(chapter_number) + "_" + str(chunked_text.index(chunk)) + ".txt")
            if not Path(filename).exists():
                print("processing chunk " + str(chunked_text.index(chunk)) + " out of " + str(len(chunked_text)))
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": chunk},
                    ]
                )
                # write the value in response['choices'][0]['message']['content'] to the file with the name chapter_name_i.csv where i is the index of the chunk
                with open(filename, "w", encoding='utf-8') as f:
                    f.write(response['choices'][0]['message']['content'])
            else:
                print("The file " + str(filename) + " already exists so skipping it")
  




processing chapter:  25 from page 178 to page 183
processing chunk 0 out of 4
processing chunk 1 out of 4
processing chunk 2 out of 4
processing chunk 3 out of 4
processing chapter:  26 from page 184 to page 193
processing chunk 0 out of 6
processing chunk 1 out of 6
processing chunk 2 out of 6
processing chunk 3 out of 6
processing chunk 4 out of 6
processing chunk 5 out of 6
processing chapter:  27 from page 194 to page 203
processing chunk 0 out of 6
processing chunk 1 out of 6
processing chunk 2 out of 6
processing chunk 3 out of 6
processing chunk 4 out of 6
processing chunk 5 out of 6
processing chapter:  28 from page 204 to page 208
processing chunk 0 out of 3
processing chunk 1 out of 3
processing chunk 2 out of 3
processing chapter:  29 from page 209 to page 212
processing chunk 0 out of 3
processing chunk 1 out of 3
processing chunk 2 out of 3
processing chapter:  30 from page 213 to page 218
processing chunk 0 out of 4
processing chunk 1 out of 4
processing chunk 2 out of 4


In [57]:
input_folder = "./summary_details"
output_folder = "./summary_chapter"
# create the Path object and the folder if it doesn't exist
folder_path = Path(output_folder)
folder_path.mkdir(parents=True, exist_ok=True)
#folder_path = Path(output_folder)

chapter_range_start = 25
chapter_range_end = 32

system_prompt = "A book chapter has been divided into sections and each section summarised. You need to combine these summaries into a single summary of the chapter."

for i in range(chapter_range_start, chapter_range_end):
        print(i)
        chapter_number = i
        print("processing chapter: ", str(chapter_number))
        chapter_summary = ""
        # search the input folder for all files that start with "chapter_" + str(chapter_number) and concatenate them into a single string
        for file in Path(input_folder).glob("chapter_" + str(chapter_number) + "_" + "*" + ".txt"):
            with open(file, "r", encoding='utf-8') as f:
                print("reading file : " + str(file))
                # Concatenate the f.read() string to chapter_summary
                chapter_summary = chapter_summary + f.read() + "\n"

        print(chapter_summary)
        # print("**********")

        #create a filename using folder_pah + "chapter" + chapter_number + "_" + index of chunk + ".csv"
        filename = folder_path / ("chapter_" + str(chapter_number) + ".txt")
        if not Path(filename).exists():
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": chapter_summary},
                ]
            )
            # write the value in response['choices'][0]['message']['content'] to the file with the name chapter_name_i.csv where i is the index of the chunk
            with open(filename, "w", encoding='utf-8') as f:
                f.write(response['choices'][0]['message']['content'])
        else:
            print("The file " + str(filename) + " already exists so skipping it")
  

25
processing chapter:  25
reading file : summary_details\chapter_25_0.txt
reading file : summary_details\chapter_25_1.txt
reading file : summary_details\chapter_25_2.txt
reading file : summary_details\chapter_25_3.txt
This extract describes how the author, Andre de Ruyter, had to adjust to living with extensive security arrangements while leading Eskom. Despite the extra precautions, de Ruyter enjoyed interacting with regular South Africans, who often encouraged him to persevere in his role. De Ruyter's bodyguards carefully planned his social engagements and checked locations before he arrived. The extract also recounts the gratitude de Ruyter felt when he had the chance to do something positive for people's lives, whether customers or employees at Eskom.
The author received an email from an 11-year-old schoolgirl asking for his thoughts on green energy. Despite being a busy CEO, he replied and addressed her formally. Months later, the girl's father, who was the general manager at Mit

In [29]:
output_folder = "./summary_chapter"
# Read all the files in the output folder and combine them into a single string
book_summary = ""
for file in Path(output_folder).glob("chapter_*"):
    with open(file, "r", encoding='utf-8') as f:
        # Concatenate the f.read() string to chapter_summary
        book_summary = book_summary + f.read() + "\n"



In [30]:
#print the number of words in book_summary
print("The number of words in the book summary is: ", len(book_summary.split(" ")))

The number of words in the book summary is:  3443


In [64]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
num_tokens = len(encoding.encode(book_summary))
print("The number of tokens in the book summary is: ", num_tokens)

The number of tokens in the book summary is:  6944


In [80]:
import tiktoken

def create_string_list(input_folder, threshold):
    # Initialize tokenizer
    #tokenizer = Tokenizer(WordLevel())
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Initialize variables
    texts = []
    current_text = ''
    current_token_count = 0

    # tmp = 0

    # Loop over each file in the directory
    for i in range(1, 32):  # The range is from 1 to 31 inclusive
        file_path = Path(input_folder) / f'chapter_{i}.txt'

        with open(file_path, 'r', encoding="utf-8") as file:
            file_text = file.read()

            # Count tokens in the file's text
            new_token_count = len(encoding.encode(file_text))
            # tmp = tmp + new_token_count
            # print("File " + str(file_path) + " is being processed. Total length in tokens is: " + str(tmp) )

            # Check if adding this text exceeds the threshold
            if current_token_count + new_token_count > threshold:
                # If it does, append the current text to the list and start a new one
                texts.append(current_text)
                current_text = file_text
                current_token_count = new_token_count
            else:
                # If not, add this text to the current text
                current_text += '\n' + file_text
                current_token_count += new_token_count

    # Append the last text if it's not empty
    if current_text:
        texts.append(current_text)

    return texts


In [86]:
all_text = create_string_list("./summary_chapter", 2200) # 2200 is just trial and error to get approximately even number of tokens per list item
print(len(all_text))
for i in range(len(all_text)):
    print(len(encoding.encode(all_text[i])))


2
2028
2021


In [95]:

input_folder = "./summary_chapter"
output_folder = "./summary_book"
# create the Path object and the folder if it doesn't exist
folder_path = Path(output_folder)
folder_path.mkdir(parents=True, exist_ok=True)
#folder_path = Path(output_folder)

system_prompt = "The book \"Truth to Power\" has been summarised by chapter. You need to combine the summary of several chapters into a single summary so we can eventually arrive at a summary for the entire book."

book_summary = []
token_limit = 3000

all_text_summary = []

for i in range(len(all_text)):    
    summary_parse_1 = ""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": all_text[i]},
        ]
    )
    r = response['choices'][0]['message']['content']
    print(r)
    print("************************")
    summary_parse_1 = summary_parse_1 + r + "\n"
    all_text_summary.append(r)



The book "Truth to Power" features several chapters that shed light on the challenges and corruption faced by South Africa's state-owned power company, Eskom. The author, a former CEO of Eskom, describes their personal experience applying and working in the role and the importance of having selfless leadership. The chapters touch on issues such as operational culture, corruption, and procurement regulations that have contributed to Eskom's struggles. Furthermore, the book highlights the efforts of key individuals in uncovering corruption and fraud at Eskom and the need for strong, decisive leadership to address the country's challenges.
************************
The book "Truth to Power" discusses the challenges facing South Africa in its transition to green energy, including the integration of energy and environmental policies, funding for green energy projects, and the need for proactive government investment. The book also explores the issues surrounding Eskom, the state-owned electr

In [99]:
# print(summary_parse_1)
system_prompt = "The book \"Truth to Power\" has been summarised in sections. You need to combine these summaries into a single summary of entire book."

# set user_content to the concatenation of all the items in the all_text_summary list separate with a newline character
user_content = ""
for i in range(len(all_text_summary)):
    user_content = user_content + all_text_summary[i] + "\n"


filename = folder_path / ("book_summary.txt")
if not Path(filename).exists():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
        ]
    )
    # write the value in response['choices'][0]['message']['content'] to the file with the name chapter_name_i.csv where i is the index of the chunk
    with open(filename, "w", encoding='utf-8') as f:
        f.write(response['choices'][0]['message']['content'])
else:
    print("The file " + str(filename) + " already exists so skipping it")
